In [15]:
#!pip install pymongo pandas
#
from pymongo import MongoClient
import pandas as pd
from datetime import datetime

# Mapeamento dos IDs dos nodos para seus sensores e variáveis correspondentes
sensor_map = {
    '124b002281ff46': 'H2S',       # ppm
    '124b002e845ca0': 'CH4',       # %
    '124b002f40ce97': 'NH3',       # ppm
    '124b002e845ca0_Temp': 'Temperatura',  # °C
    '124b002e845ca0_Hum': 'Humidade',      # %
    '124b002e846087': 'CO2',       # ppm
    '124b002e845f31': 'NO2'        # ppm
}

# Funções de conversão de sinal para unidades apropriadas
def convert_signal(sensor_name, signal):
    if sensor_name == 'H2S' or sensor_name == 'NH3' or sensor_name == 'NO2' or sensor_name == 'CO2':
        # Exemplo de conversão para ppm
        # Supondo que o sinal é proporcional e vai de -100 dBm (0 ppm) a 0 dBm (máx ppm)
        # Ajuste a função conforme a calibração específica do sensor
        return max(0, (100 + signal) * (100 if sensor_name == 'H2S' else 5000 if sensor_name == 'CO2' else 100))
    elif sensor_name == 'CH4':
        # Exemplo de conversão para % de metano
        return max(0, (100 + signal) * 10 / 100)
    elif sensor_name == 'Temperatura':
        # Exemplo de conversão para graus Celsius (assumindo -100 dBm = -45°C, 0 dBm = 125°C)
        return max(-45, (100 + signal) * 170 / 100 - 45)
    elif sensor_name == 'Humidade':
        # Exemplo de conversão para % de humidade
        return max(0, (100 + signal) * 100 / 100)
    else:
        return signal  # Retorna o sinal bruto se não for reconhecido

# URL de conexão ao MongoDB
mongo_url = "mongodb://179.124.146.5:27017/"

# Conectar ao MongoDB
client = MongoClient(mongo_url)

# Acessar o banco de dados "Doutorado"
db = client["Doutorado"]

# Acessar a coleção "Sensores"
collection = db["Sensores"]

# Limitar a quantidade de registros a serem exibidos
max_records = 100

# Lista para armazenar os registros formatados
records = []

# Buscar e processar os documentos
for document in collection.find().limit(max_records):
    sensor_name = sensor_map.get(document.get('id'), 'Unknown Sensor')

    # Processar e formatar o status
    for status in document.get('status', []):
        converted_signal = convert_signal(sensor_name, status['signal'])
        formatted_record = {
            'ID': document.get('id'),
            'Sensor Name': sensor_name,
            'Timestamp': datetime.fromtimestamp(status['timestamp']).strftime('%d/%m/%Y %H:%M:%S'),
            'Converted Value': converted_signal
        }
        records.append(formatted_record)

# Converter a lista de registros para um DataFrame do pandas
df = pd.DataFrame(records)

# Exportar para um arquivo CSV
df.to_csv("sensoresDataset_converted.csv", index=False, encoding='utf-8-sig')

# Exibir as primeiras linhas do DataFrame convertido
df.head()




,ID,Sensor Name,Timestamp,Converted Value
0,124b002e845f31,NO2,31/07/2024 10:30:53,3500
1,124b002281ff46,H2S,31/07/2024 10:30:34,1900
2,124b002e846087,CO2,31/07/2024 10:32:00,200000
3,124b002e845f4d,Unknown Sensor,31/07/2024 10:29:24,-65
4,124b002f40ce97,NH3,31/07/2024 10:29:08,3000
